In [ ]:
import keras
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense, Activation

In [ ]:
import numpy as np
from multigraph import MultiGraph

In [ ]:
#import csv
import unicodecsv as csv
""" EXAMPLE
add,IS,operator
subtract,IS,operator
multiply,IS,operator
divide,IS,operator
open_closure,IS,operator
close_closure,IS,operator
"""
op_graph = MultiGraph()
#with open('operator_graph.csv', 'r') as csvfile:
n = 45
for i in range(n):
    print '.',
    with open('aifb_csv/aifb_relation_'+str(i)+'.csv', 'r') as csvfile:
        graphreader = csv.reader(csvfile, delimiter=",")
        for row in graphreader:
            op_graph.add_connection(row)
print('\n loaded '+str(op_graph.n_rels)+' relations.')



In [ ]:
a_graph = op_graph.get_adjacency_matrix()
print("number of nodes",op_graph.n_nodes)
print("number of relations",op_graph.n_rels)
print("relations:")
for k,v in op_graph.rel_counter.iteritems():
    print(k.rsplit('/',1)[-1], v)

In [ ]:
x_train = a_graph
print('input dims:',x_train.shape)
print(type(x_train))
sum(list(x_train[5]))

In [ ]:
# Need to make a custom Dense layer to tie weights between the encoder and decoder
from keras import backend as K

class DenseTied(Dense):
    def __init__(self, master_layer, **kwargs):
        #output_dim needs to be equal to the input dimensions of the master_layer
        self.output_dim = master_layer.input_shape[-1]
        super(DenseTied, self).__init__(self.output_dim, **kwargs)
        self.master_layer = master_layer
    
    def build(self,input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]

        self.kernel = K.transpose(self.master_layer.kernel)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.output_dim,),
                            initializer=self.bias_initializer,
                            name='bias',
                            regularizer=self.bias_regularizer,
                            constraint=self.bias_constraint)
        else:
            self.bias = None
        self.built = True
        
    def call(self, inputs):
        output = K.dot(inputs, K.transpose(self.master_layer.kernel))
        if self.use_bias:
            output = K.bias_add(output, self.bias, data_format='channels_last')
        if self.activation is not None:
            output = self.activation(output)
        return output


In [ ]:
encoding_dim = 128
input_dim = x_train.shape[1]

inputs = Input(shape=(input_dim,),name="inputs")
# Encoder Layers
encode_layer1 = Dense(4 * encoding_dim, activation='tanh',name="encode_layer1")
encode_layer2 = Dense(2 * encoding_dim, activation='tanh',name="encode_layer2")
coding_layer = Dense(encoding_dim, activation='tanh',name="coding_layer")

encoding_1 = encode_layer1(inputs)
encoding_2 = encode_layer2(encoding_1)
the_code = coding_layer(encoding_2)

# Decoder Layers
#decode_layer1 = Dense(2 * encoding_dim, activation='tanh',name="decode_layer1")
#decode_layer2 = Dense(4 * encoding_dim, activation='tanh',name="decode_layer2")
#reconstruction_layer = Dense(input_dim, activation='tanh',name="reconstruction_layer")
decode_layer1 = DenseTied(coding_layer, activation='tanh',name="decode_layer1")
decode_layer2 = DenseTied(encode_layer2, activation='tanh',name="decode_layer2")
recon_layer = DenseTied(encode_layer1, activation='tanh',name="reconstruction_layer")

decoding_1 = decode_layer1(the_code)
decoding_2 = decode_layer2(decoding_1)
reconstruction = recon_layer(decoding_2)

ae = Model(inputs=inputs, outputs=reconstruction)
#monitor = EarlyStopping(monitor='loss', min_delta=0.0001, patience=5, verbose=1, mode='auto')
ae.compile(optimizer='adam', loss='mse')
ae.summary()

In [ ]:
callbacks = []
DEBUG = False

class WeightHistory(keras.callbacks.Callback):
    def on_batch_end(self, batch, logs={}):
        print(batch)
        print('coding layer weights')
        print(self.model.get_layer('coding_layer').get_weights())
        print(self.model.get_layer('coding_layer').get_weights()[0].shape)
        print('decoding layer weights')
        print(np.transpose(self.model.get_layer('decode_layer1').get_weights()))
        print(np.transpose(self.model.get_layer('decode_layer1').get_weights()[0]).shape)


history = WeightHistory()

if DEBUG:
    callbacks.append(history)


In [ ]:
ae.fit(x_train, x_train.toarray(), epochs=2, verbose=1,callbacks=callbacks)

In [ ]:
#coding_model = Model(inputs=ae.inputs, outputs=ae.get_layer("the_code").output)
coding_model = Model(inputs=inputs, outputs=the_code)

In [ ]:
#save the embeddings in order to plot them later
# serialize model to JSON
model_json = coding_model.to_json()
with open("coding_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
coding_model.save_weights("coding_model.h5")
print("Saved model to disk")

In [ ]:
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

#save the adjacency matrix as well
save_object(op_graph, 'adj_graph.pkl')
print("Saved graph object to disk")